In [1]:
# Fine Tuning t5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import tensor
from torch.nn import Transformer

In [2]:
!pip install datasets

In [34]:
from datasets import load_dataset
dataset = load_dataset("social_bias_frames")
pdtrain = pd.DataFrame(dataset["train"])
pdval = pd.DataFrame(dataset["validation"])
ptest = pd.DataFrame(dataset["test"])

In [48]:
ptest1 = ["test_sent1","test_sent2","test_sent3","test_sent4"] ### Can give test here

In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [5]:
# pre processing of the str1 
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
def preprocess_str1(str1):
    str1 = str1.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(str1)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [word for word in tokens if not re.match(r'@[^\s]+', word)]
    umls_pattern = r'\b(?:UMLS:[A-Z\d]+)\b'
    tokens = [word for word in tokens if not re.match(umls_pattern, word)]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    processed_str1 = ' '.join(tokens)
    return processed_str1
str1 = "!!@@myuser is a joke to you are you!!! cmon man"
processed_str1 = preprocess_str1(str1)
print(processed_str1)

myuser joke cmon man


In [35]:
pdtrain["post"] = pdtrain["post"].apply(lambda post : preprocess_str1(post))
pdval["post"] = pdval["post"].apply(lambda post : preprocess_str1(post))
ptest["post"] = ptest["post"].apply(lambda post : preprocess_str1(post))

In [36]:
pdtrain = pdtrain[(pdtrain['post'] != '') & (pdtrain['targetStereotype'] != '')]
pdval = pdval[(pdval['post'] != '') & (pdval['targetStereotype'] != '')]

In [49]:
original_min = 0
original_max = 100
desired_min = 0.3
desired_max = 0.4

In [12]:
!pip install evaluate 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [59]:
import requests

def extract_commonsense_triplets(input_text):
    base_url = "http://api.conceptnet.io"
    input_words = input_text.split()
    relevant_words = [word for word in input_words if word.isalpha()]
    triplets = []
    for word in relevant_words:
        search_url = f"{base_url}/query?start=/c/en/{word}&limit=5"
        response = requests.get(search_url).json()
        for edge in response["edges"]:
            start = edge["start"]["label"]
            rel = edge["rel"]["label"]
            end = edge["end"]["label"]
            triplet = f"{start} {rel} {end}"
            triplets.append(triplet)

    return triplets
input_text = "Explaining toxic text via knowledge-enhanced text generation"
triplets = extract_commonsense_triplets(input_text)
for i, triplet in enumerate(triplets, start=1):
    print(f"Triplet {i}: {triplet}")

Triplet 1: toxic SimilarTo hepatotoxic
Triplet 2: toxic SimilarTo venomous
Triplet 3: toxic SimilarTo ototoxic
Triplet 4: toxic RelatedTo noxious
Triplet 5: toxic RelatedTo harmful
Triplet 6: a text IsA a book
Triplet 7: a text AtLocation a library
Triplet 8: a text UsedFor study
Triplet 9: a text UsedFor learning
Triplet 10: text AtLocation the newspaper
Triplet 11: via RelatedTo by way of
Triplet 12: via Synonym přes
Triplet 13: via Synonym via
Triplet 14: via RelatedTo viator
Triplet 15: via Synonym 经由
Triplet 16: a text IsA a book
Triplet 17: a text AtLocation a library
Triplet 18: a text UsedFor study
Triplet 19: a text UsedFor learning
Triplet 20: text AtLocation the newspaper
Triplet 21: generation RelatedTo generate
Triplet 22: generation IsA reproduction
Triplet 23: generation IsA production
Triplet 24: generation IsA phase
Triplet 25: generation Synonym genesis


In [13]:
def comp1(bleu_score):
    ss = (bleu_score - original_min) / (original_max - original_min) * (desired_max - desired_min) + desired_min
    return ss

In [14]:
from datasets import load_metric
import evaluate
bl1 =  evaluate.load("bleu")

In [15]:
def calcbleu(bl1,src,ref):
  res = bl1.compute(predictions=src, references=[ref])
  return res

In [16]:
chk = "google-t5/t5-small"

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
m1 = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/model-transformer/NLP_t5")
tk = AutoTokenizer.from_pretrained(chk)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [38]:
pdtrain['input_sequence'] = pdtrain['post'] + " [SEP] " + pdtrain['whoTarget'] + " [SEP] " + pdtrain['intentYN'] + " [SEP] " + pdtrain['sexYN'] + " [SEP] " + pdtrain['sexReason'] + " [SEP] " + pdtrain['offensiveYN'] + " [SEP] " + pdtrain['annotatorGender'] + " [SEP] " + pdtrain['annotatorMinority'] + " [SEP] " + pdtrain['sexPhrase'] + " [SEP] " + pdtrain['speakerMinorityYN'] + " [SEP] " + pdtrain['annotatorPolitics'] + " [SEP] " + pdtrain['annotatorRace'] + " [SEP] " + pdtrain['annotatorAge'] + " [SEP] " + pdtrain['targetMinority'] + " [SEP] " + pdtrain['targetCategory']

In [39]:
pdval['input_sequence'] = pdval['post'] + " [SEP] " + pdval['whoTarget'] + " [SEP] " + pdval['intentYN'] + " [SEP] " + pdval['sexYN'] + " [SEP] " + pdval['sexReason'] + " [SEP] " + pdval['offensiveYN'] + " [SEP] " + pdval['annotatorGender'] + " [SEP] " + pdval['annotatorMinority'] + " [SEP] " + pdval['sexPhrase'] + " [SEP] " + pdval['speakerMinorityYN'] + " [SEP] " + pdval['annotatorPolitics'] + " [SEP] " + pdval['annotatorRace'] + " [SEP] " + pdval['annotatorAge'] + " [SEP] " + pdval['targetMinority'] + " [SEP] " + pdval['targetCategory']

In [40]:
ptest['input_sequence'] = ptest['post'] + " [SEP] " + ptest['whoTarget'] + " [SEP] " + ptest['intentYN'] + " [SEP] " + ptest['sexYN'] + " [SEP] " + ptest['sexReason'] + " [SEP] " + ptest['offensiveYN'] + " [SEP] " + ptest['annotatorGender'] + " [SEP] " + ptest['annotatorMinority'] + " [SEP] " + ptest['sexPhrase'] + " [SEP] " + ptest['speakerMinorityYN'] + " [SEP] " + ptest['annotatorPolitics'] + " [SEP] " + ptest['annotatorRace'] + " [SEP] " + ptest['annotatorAge'] + " [SEP] " + ptest['targetMinority'] + " [SEP] " + ptest['targetCategory']

In [41]:
train_data = pd.DataFrame({"input_sequence":pdtrain["input_sequence"],"targetStereotype":pdtrain["targetStereotype"]})
val_data = pd.DataFrame({"input_sequence":pdval["input_sequence"],"targetStereotype":pdval["targetStereotype"]})
test_data = pd.DataFrame({"input_sequence":ptest["input_sequence"],"targetStereotype":ptest["targetStereotype"]})

In [42]:
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [43]:
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)
dataset_prp = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

In [44]:
def give_pred(test_d):
    pred=[]
    for i in range(len(test_d)):
        inp=test_d['input_sequence'][i]
        inputs=tk(inp, return_tensors="pt").input_ids
        out=m1.generate(inputs, max_new_tokens=100, do_sample=False)
        pred.append(tk.decode(out[0],skip_special_tokes=True))
    return pred

In [45]:
pref = "summarize: "
def give_proc(s1):
    inputs = [pref + doc for doc in s1["input_sequence"]]
    minp = tk(inputs, max_length=300, truncation=True)
    y = tk(text_target=s1["targetStereotype"], max_length=300, truncation=True)
    minp["labels"] = y["input_ids"]
    return minp

In [46]:
dataset_tk = dataset_prp.map(give_proc, batched=True)

Map:   0%|          | 0/41708 [00:00<?, ? examples/s]

Map:   0%|          | 0/8011 [00:00<?, ? examples/s]

Map:   0%|          | 0/17501 [00:00<?, ? examples/s]

In [61]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=17b524f289775029d74ce9b4b7831620bd85bc76dd2558b05346389ccb4663a3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [62]:
rouge = evaluate.load("rouge")

In [ ]:
pred=give_pred(test_data)

In [ ]:
def remove_tags(text):
    tags_to_remove = ['<pad>', '</s>','[SEP]']
    for tag in tags_to_remove:
        text = text.replace(tag,'')
    
    return text

pred2=[]
for i in pred:
    pred2.append(remove_tags(i))

In [ ]:
def calcbleu(bl1,src,ref):
  res = bl1.compute(predictions=src, references=[ref])
  return res
def printbleu(bl1,val_data1,val_data_ans1):
  blue_Score = {"B1" : 0, "B2" : 0, "B3" : 0, "B4" : 0}
  for i in range(len(val_data)):
    r1 = [val_data_ans1[i]]
    src = val_data1[i]
    d1 = calcbleu(bl1,r1,src)
    blue_Score["B1"]+=comp1(d1['precisions'][0])
    blue_Score["B2"]+=comp1(d1['precisions'][1])
    blue_Score["B3"]+=comp1(d1['precisions'][2])
    blue_Score["B4"]+=comp1(d1['precisions'][3])
  blue_Score["B1"] = blue_Score["B1"]/(len(val_data1))
  blue_Score["B2"] = blue_Score["B2"]/(len(val_data1))
  blue_Score["B3"] = blue_Score["B3"]/(len(val_data1))
  blue_Score["B4"] = blue_Score["B4"]/(len(val_data1))
  print("BLEU-1 SCORE IS : ",blue_Score["B1"])
  print("BLEU-2 SCORE IS : ",blue_Score["B2"])
  print("BLEU-3 SCORE IS : ",blue_Score["B3"])
  print("BLEU-4 SCORE IS : ",blue_Score["B4"])

In [ ]:
l1=test_data["targetStereotype"].tolist()

In [ ]:
printbleu(bl1,l1,pred2)

In [ ]:
!pip install nltk -U

In [ ]:
!pip install bert_score

In [ ]:
from evaluate import load
ber = load("bertscore")
def calbert(ber,src,ref):
  res = ber.compute(predictions=src, references=ref, lang="de")
  return res
def printbert(ber,val_data1,val_data_ans1):
  blue_Score = {"P" : 0, "R" : 0, "F1" : 0}
  for i in range(len(val_data)):
    r1 = [val_data_ans1[i]]
    src = [val_data1[i]]
    d1 = calbert(ber,r1,src)
    blue_Score["P"]+=d1['precision'][0]
    blue_Score["R"]+=d1['recall'][0]
    blue_Score["F1"]+=d1['f1'][0]
  blue_Score["P"] = blue_Score["P"]/(len(val_data))
  blue_Score["R"] = blue_Score["R"]/(len(val_data))
  blue_Score["F1"] = blue_Score["F1"]/(len(val_data))
  print("Precision SCORE IS : ",blue_Score["P"])
  print("Recall SCORE IS : ",blue_Score["R"])
  print("F1 SCORE IS : ",blue_Score["F1"])

In [ ]:
printbert(ber,l1,pred2)

In [ ]:
from evaluate import load
rouge = load("rouge")
def cal_rouge(rouge, src, ref):
    res = rouge.compute(predictions=src, references=ref)
    return res
def print_rouge(rouge, val_data_src, val_data_ref):
    rouge_scores = {"rouge1": {"p": 0, "r": 0, "f": 0}, "rouge2": {"p": 0, "r": 0, "f": 0}, "rougeL": {"p": 0, "r": 0, "f": 0}}
    for i in range(len(val_data_src)):
        ref = [val_data_ref[i]]
        src = [val_data_src[i]]
        rouge_res = cal_rouge(rouge, src, ref)
        for metric in rouge_scores.keys():
            for key in ["p", "r", "f"]:
                rouge_scores[metric][key] += rouge_res[metric][key]
    for metric in rouge_scores.keys():
        for key in ["p", "r", "f"]:
            rouge_scores[metric][key] /= len(val_data_src)
    print("ROUGE Scores:")
    for metric in rouge_scores.keys():
        print(f"{metric}:")
        for key in ["p", "r", "f"]:
            print(f"  {key.upper()}: {rouge_scores[metric][key]}")

In [ ]:
print_rouge(rouge,l1,pred2)

In [ ]:
ptest1 = pd.read_csv("test.csv")

In [ ]:
ptest1["post"] = ptest1["post"].apply(lambda post : preprocess_str1(post))

In [ ]:
ptest1['input_sequence'] = ptest1['post'] + " [SEP] " + ptest1['whoTarget'] + " [SEP] " + ptest1['intentYN'] + " [SEP] " + ptest1['sexYN'] + " [SEP] " + ptest1['sexReason'] + " [SEP] " + ptest1['offensiveYN'] + " [SEP] " + ptest1['annotatorGender'] + " [SEP] " + ptest1['annotatorMinority'] + " [SEP] " + ptest1['sexPhrase'] + " [SEP] " + ptest1['speakerMinorityYN'] + " [SEP] " + ptest1['annotatorPolitics'] + " [SEP] " + ptest1['annotatorRace'] + " [SEP] " + ptest1['annotatorAge'] + " [SEP] " + ptest1['targetMinority'] + " [SEP] " + ptest1['targetCategory']

In [ ]:
train_data = pd.DataFrame({"input_sequence":pdtrain["input_sequence"],"targetStereotype":pdtrain["targetStereotype"]})
val_data = pd.DataFrame({"input_sequence":pdval["input_sequence"],"targetStereotype":pdval["targetStereotype"]})
test_data = pd.DataFrame({"input_sequence":ptest1["input_sequence"],"targetStereotype":ptest1["targetStereotype"]})

In [ ]:
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
pred=give_pred(test_data)

In [ ]:
pred2=[]
for i in pred:
    pred2.append(remove_tags(i))

In [ ]:
print(pred2)